In [11]:
import diffuser.utils as utils
import ipdb
from trajdata import AgentBatch, AgentType, UnifiedDataset
from torch.utils.data import DataLoader
import os
from diffuser.utils.training_ped import Trainer
from ldm.models.diffusion.ddpm import Diffusion
import time
from ldm.models.diffusion.ddim import DDIMSampler
import torch
from trajdata_utils.interactive_vis import plot_agent_batch_interactive
from ldm.models.diffusion.diffusion import GaussianDiffusion
from ldm.modules.diffusionmodules.trajunet import UNetModel
import copy

In [32]:
#-----------------------------------------------------------------------------#
#---------------------------------- dataset ----------------------------------#
#-----------------------------------------------------------------------------#

dataset = UnifiedDataset(
    desired_data=["eupeds_eth", "eupeds_hotel", "eupeds_univ", "eupeds_zara1", "eupeds_zara2"],
    data_dirs={  # Remember to change this to match your filesystem!
        "eupeds_eth": "~/traj_dataset/eth-uty",
        "eupeds_hotel": "~/traj_dataset/eth-uty",
        "eupeds_univ": "~/traj_dataset/eth-uty",
        "eupeds_zara1": "~/traj_dataset/eth-uty",
        "eupeds_zara2": "~/traj_dataset/eth-uty",
    },
    history_sec =(0.0,  0.0), 
    future_sec =(15.9,  15.9),
    desired_dt=0.1
)

dataloader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=True,
    collate_fn=dataset.get_collate_fn(),
    num_workers=0, # This can be set to 0 for single-threaded loading, if desired.
)


In [35]:
#-----------------------------------------------------------------------------#
#-------------------------------- instantiate --------------------------------#
#-----------------------------------------------------------------------------#

horizon = 160

unet = UNetModel(
    horizon = horizon
).cuda()

diffusion = GaussianDiffusion(
    model = unet,
    horizon = horizon
).cuda()

timestr = time.strftime("%Y%m%d-%H%M")
output_path = "./logs/eupeds/"+timestr
trainer = Trainer(
    diffusion_model=diffusion,
    data_loader=dataloader,
    results_folder=output_path
)

[ models/temporal ] Channel dimensions: [(4, 32), (32, 64), (64, 128), (128, 256)]
[(4, 32), (32, 64), (64, 128), (128, 256)]


In [34]:
# trainer.load(0,'../logs/eupeds/20231114-1826/latest.pt')
trainer.load(0,'../logs/eupeds/diffuser/latest.pt')
# sampler = DDIMSampler(trainer.model)

In [14]:
data = iter(dataloader)
batch = next(data)

In [15]:
pos = torch.cat((batch.agent_hist.position, batch.agent_fut.position), axis=1).transpose(1,2).cuda()
vel = torch.cat((batch.agent_hist.velocity, batch.agent_fut.velocity), axis=1).transpose(1,2).cuda()

pos = pos/10 - 0.5
vel = vel/10
traj = torch.cat((pos, vel), axis=1)

cond = pos[:,:2,(0,-1)]

In [7]:

traj_gen = diffusion.conditional_sample(cond)

traj_gen[:,:2,:] = (traj_gen[:,:2,:]+0.5)*10
traj_gen[:,2:4,:] = traj_gen[:,2:4,:]*10

In [21]:
traj_gen = traj_gen.transpose(1,2)

In [21]:
plot_agent_batch_interactive(batch, batch_idx=0, cache_path=dataset.cache_path)

In [23]:
batch_gen = copy.deepcopy(batch)
batch_gen.agent_hist.position = traj_gen[:,0,:2].cpu()
batch_gen.agent_fut.position = traj_gen[:,1:,:2].cpu()
batch_gen.agent_hist.velocity = traj_gen[:,0,2:].cpu()
batch_gen.agent_fut.velocity = traj_gen[:,1:,2:].cpu()

In [24]:
plot_agent_batch_interactive(batch_gen, batch_idx=0, cache_path=dataset.cache_path)

## CBF weighted MAP

In [26]:
batch = next(data)
pos = torch.cat((batch.agent_hist.position, batch.agent_fut.position), axis=1).transpose(1,2).cuda()
vel = torch.cat((batch.agent_hist.velocity, batch.agent_fut.velocity), axis=1).transpose(1,2).cuda()

pos = pos/10 - 0.5
vel = vel/10
traj = torch.cat((pos, vel), axis=1)

cond = pos[:,:2,(0,-1)]

In [41]:
pos_neigh = torch.cat((batch.agent_hist.position+batch.neigh_hist.position, batch.agent_fut.position+batch.neigh_fut.position), axis=2).transpose(2,3).cuda()

In [28]:
pos_neigh.shape

torch.Size([1, 49, 2, 160])

In [29]:
rad = 1

rad = rad/10
traj_gen = diffusion.weight_sample(cond, pos_neigh, rad, s=0.1)

In [30]:
traj_gen[:,:2,:] = (traj_gen[:,:2,:]+0.5)*10
traj_gen[:,2:4,:] = traj_gen[:,2:4,:]*10
traj_gen = traj_gen.transpose(1,2)

In [31]:
plot_agent_batch_interactive(batch, batch_idx=0, cache_path=dataset.cache_path)
batch_gen = copy.deepcopy(batch)
batch_gen.agent_hist.position = traj_gen[:,0,:2].cpu()
batch_gen.agent_fut.position = traj_gen[:,1:,:2].cpu()
batch_gen.agent_hist.velocity = traj_gen[:,0,2:].cpu()
batch_gen.agent_fut.velocity = traj_gen[:,1:,2:].cpu()
plot_agent_batch_interactive(batch_gen, batch_idx=0, cache_path=dataset.cache_path)

In [36]:
cond.shape

torch.Size([1, 2, 2])

In [39]:
cond = torch.tensor([[[-0.5, 0], [-0.5, 0]]]).cuda()

In [52]:
pos_neigh[pos_neigh.isnan()] = torch.inf
dx = traj[:,0,:]-pos_neigh[:,:,0,:]
dy = traj[:,1,:]-pos_neigh[:,:,1,:]

In [53]:
dis = dx**2 + dy**2

In [54]:
dis.shape

torch.Size([1, 49, 160])

In [56]:
min_idx = torch.min(dis, axis=1).indices

In [57]:
min_idx.shape

torch.Size([1, 160])